In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv

c:\Users\areef\anaconda3\envs\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
disease=input("Enter your Question or disease:\n")

In [3]:
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')

In [4]:
loader=PyPDFLoader(r"C:\Users\areef\Desktop\Gen AI\pdfs\medical test book 1.pdf")
data=loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=5)
all_splits = text_splitter.split_documents(data)

In [40]:
len(all_splits)

39699

In [ ]:
gemini_embedding = GoogleGenerativeAIEmbeddings(google_api_key = GOOGLE_API_KEY, model = "models/embedding-001")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=gemini_embedding)

In [59]:
prompt2 = f"Tell me about the medicine for {disease}. Provide detailed information including uses, dosage, side effects, and precautions."

In [44]:
retrieval=vectorstore.as_retriever()
docs=retrieval.get_relevant_documents(disease)

In [35]:
print(docs)

[Document(metadata={'page': 210, 'source': 'C:\\Users\\areef\\Desktop\\Gen AI\\pdfs\\medical test book 1.pdf'}, page_content='the vasomotor center are activated and vasoconstriction commences. The individual \nfirst notices vasoconstriction in the hands and feet. Shunting of blood away from the \nperiphery to the internal organs essentially decreases heat loss from the skin, and the \nperson feels cold. For most fevers, body temperature increases by 1 to 2°C. Shivering, \nwhich increases heat production from the muscles, may begin at this time; however,'), Document(metadata={'page': 210, 'source': 'C:\\Users\\areef\\Desktop\\Gen AI\\pdfs\\medical test book 1.pdf'}, page_content='FEVER VERSUS HYPERTHERMIA\nFEVER\nFever is an elevation of body temperature that exceeds the normal daily variation and \noccurs in conjunction with an increase in the hypothalamic set point -- for example, from \n37°C to 39°C. This shift of the set point from "normothermic" to febrile levels very much \nresemb

In [33]:
model=ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True, api_key=GOOGLE_API_KEY,temperature = 0.1)

In [36]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [45]:
input_mess=ChatPromptTemplate.from_messages([("system",prompt_template),("user","{question}")])
mess=input_mess.invoke({"context":docs,"question":disease})
response=model.invoke(mess)

c:\Users\areef\anaconda3\envs\env\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [46]:
parser=StrOutputParser()
parser.invoke(response).strip()

'Fever is an elevation of body temperature that exceeds the normal daily variation and occurs in conjunction with an increase in the hypothalamic set point—for example, from 37°C to 39°C.  This resembles resetting a thermostat to a higher level.'